# in terminal / cli

#!pip install langchain-anthropic


# Basic Setup

Here, we will test whether calling your API Models is working. 
With LangChain, one can build LLM applications. One type of LLM application you can build is an agent.  

In practice, it can be difficult to build systems that reliably execute on these tasks. As LangChain worked with their users to put agents into production, they have learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, LangChain built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## LangGraph Studio

[LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), is an IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this workshop, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host Chat Models, they rely on third party integration. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain. By default, we will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is popular and performant. please ensure that you have an `OPENAI_API_KEY`.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a how-do for all the things that you can do with chat models, but we'll show a few highlights. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [4]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

Chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [5]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Ryan")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cd9925b-e4b0-4877-a4b7-673aba63c3d2-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-98cbc139-9eed-4806-9400-2d57e555a1c4-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



## Import Other Chat Models


In [39]:
#!pip install langchain-anthropic

In [40]:
from langchain_anthropic import ChatAnthropic
#from langchain_core.messages import AIMessage, HumanMessage #already imported above

In [41]:
anthropic_chat = ChatAnthropic(model="claude-3-opus-20240229", temperature=0, max_tokens=1024)

In [42]:
_set_env("ANTHROPIC_API_KEY")

In [43]:
# Create a message
msg = HumanMessage(content="What is the capital of Taiwan?")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
anthropic_chat.invoke(messages)

AIMessage(content='The capital of Taiwan is Taipei. It is located in the northern part of the island and is the political, economic, and cultural center of Taiwan. Taipei is also the most populous city in Taiwan, with a population of approximately 2.6 million people in the city proper and over 7 million in the metropolitan area.\n\nTaipei is home to many important government institutions, including the Office of the President, the Executive Yuan, and the Legislative Yuan. The city is also known for its modern infrastructure, vibrant nightlife, and diverse cuisine. Some famous landmarks in Taipei include Taipei 101 (a skyscraper that was once the tallest building in the world), the National Palace Museum, and the Chiang Kai-shek Memorial Hall.', additional_kwargs={}, response_metadata={'id': 'msg_01AEHy1vRLWUfcuGg9iPfu3o', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_

## Search Tools

[Tavily](https://tavily.com/) is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. it's easy to sign up and offers a free tier. We will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [7]:
_set_env("TAVILY_API_KEY")

TAVILY_API_KEY:  ········


In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is Interaction Design?")

In [9]:
search_docs

[{'url': 'https://www.geeksforgeeks.org/whai-is-interaction-design/',
  'content': 'Interaction design is the process of creating digital products, such as websites and apps, that are easy and enjoyable to use. It focuses on how users interact with these products, ensuring that every click, swipe, and action feels natural and intuitive. This involves designing buttons, menus, and other interactive elements to make sure they'},
 {'url': 'https://www.interaction-design.org/literature/topics/interaction-design',
  'content': "Interaction Design (IxD) is the design of interactive products and services in which a designer's focus goes beyond the item in development to include the way users will interact with it. Thus, close scrutiny of users' needs, limitations and contexts, etc. empowers designers to customize output to suit precise demands."},
 {'url': 'https://en.wikipedia.org/wiki/Interaction_design',
  'content': 'Interaction design, often abbreviated as IxD, is "the practice of design